# Data Preparation

In [1]:
# | echo: false
%load_ext lab_black

Import Python modules

In [2]:
import os
from datetime import datetime
from glob import glob
from typing import Dict, List

import pandas as pd
import pytz
from google.oauth2 import service_account

## About

This step of the analysis will prepare data for use in exploratory and quantitative data analysis.

### Discussion of Study Period for This Project
We will need to discuss the data that can be used to prepare data without suffering from lookahead bias/data leakage.

Data splits are created in a later step. ML model development will be performed using a training data split, validation (feature selection, hyperparameter tuning, etc.) is performed using the validation data split and, finally, the best ML model is evaluated using the test data split.

So, the validation and test data splits should be treated as unseen data. In order to avoid lookahead bias/data leakage during data preparation, this means that data preparation steps should be determined using the training data and simply applied to the validation and test data splits.

### Data Selection for Data Preparation
With the above in mind, data preparation will cover the training data.

## User Inputs

Get relative path to project root directory

In [3]:
# | code-fold: false
PROJ_ROOT_DIR = os.path.join(os.pardir)

Define the following

1. train data start date
2. train data end date
3. test data end date

In [4]:
# | code-fold: false
train_start_date = "20160901"
train_end_date = "20161231"
test_end_date = "20170228"

Retrieve credentials for `bigquery` client

In [5]:
# | code-fold: false
# Google Cloud PROJECT ID
gcp_project_id = os.environ["GCP_PROJECT_ID"]

Get filepath to Google Cloud Service Account JSON key

In [6]:
# | code-fold: false
raw_data_dir = os.path.join(PROJ_ROOT_DIR, "data", "raw")
gcp_creds_fpath = glob(os.path.join(raw_data_dir, "*.json"))[0]

Authenticate `bigquery` client and get dictionary with credentials

In [7]:
# | code-fold: false
gcp_credentials = service_account.Credentials.from_service_account_file(gcp_creds_fpath)
gcp_auth_dict = dict(gcp_project_id=gcp_project_id, gcp_creds=gcp_credentials)

Create a mapping between action type integer and label, in order to get meaningful names from the action_type column

In [8]:
# | code-fold: false
mapper = {
    1: "Click through of product lists",
    2: "Product detail views",
    3: "Add product(s) to cart",
    4: "Remove product(s) from cart",
    5: "Check out",
    6: "Completed purchase",
    7: "Refund of purchase",
    8: "Checkout options",
    0: "Unknown",
}

Define a Python helper function to execute a SQL query using Google BigQuery

In [10]:
# | code-fold: false
def run_sql_query(
    query: str,
    gcp_project_id: str,
    gcp_creds: os.PathLike,
    show_dtypes: bool = False,
    show_info: bool = False,
    show_df: bool = False,
) -> pd.DataFrame:
    """Run query on BigQuery and return results as pandas.DataFrame."""
    start_time = datetime.now(pytz.timezone("US/Eastern"))
    start_time_str = start_time.strftime("%Y-%m-%d %H:%M:%S.%f")
    print(f"Query execution start time = {start_time_str[:-3]}...", end="")
    df = pd.read_gbq(
        query,
        project_id=gcp_project_id,
        credentials=gcp_creds,
        dialect="standard",
        # configuration is optional, since default for query caching is True
        configuration={"query": {"useQueryCache": True}},
        # use_bqstorage_api=True,
    )
    end_time = datetime.now(pytz.timezone("US/Eastern"))
    end_time_str = end_time.strftime("%Y-%m-%d %H:%M:%S.%f")
    duration = end_time - start_time
    duration = duration.seconds + (duration.microseconds / 1_000_000)
    print(f"done at {end_time_str[:-3]} ({duration:.3f} seconds).")
    print(f"Query returned {len(df):,} rows")
    if show_df:
        with pd.option_context("display.max_columns", None):
            display(df)
    if show_dtypes:
        display(df.dtypes.rename("dtype").to_frame().transpose())
    if show_info:
        df.info()
    return df

## Data Preparation

### Visitors Who Made a Purchase

**Question 1. Get visitors with a purchase on a future visit to the Marketplace.**

To get these visitors, a similar approach to that from the get-data step will be used. In that step, two criteria were used to identify a purchase on a future visit, namely `total.transactions > 0` and `totals.newVisits IS NULL`. Those will be used here as well.

A query with these filters is executed below

In [11]:
# | code-fold: false
query = f"""
        SELECT fullvisitorid,
               IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, True, False) AS made_purchase_on_future_visit,
               IF(
                   SUM(CASE WHEN totals.transactions > 0 AND totals.newVisits IS NULL THEN 1 ELSE 0 END) > 0, True, False
               ) AS made_purchase_on_future_visit_v2
        FROM `data-to-insights.ecommerce.web_analytics`
        WHERE date BETWEEN '{train_start_date}' AND '{test_end_date}'
        AND geoNetwork.country = 'United States'
        GROUP BY fullvisitorid
        """
df = run_sql_query(query, **gcp_auth_dict, show_df=False)
display(
    # breakdown of returning purchasers using COUNTIF()
    df["made_purchase_on_future_visit"]
    .value_counts()
    .rename("num_return_purchasers_using_countif")
    .to_frame()
    .merge(
        # breakdown of returning purchasers using CASE WHEN()
        df["made_purchase_on_future_visit_v2"]
        .value_counts()
        .rename("num_return_purchasers_using_if_sum_casewhen")
        .to_frame(),
        left_index=True,
        right_index=True,
    )
    .merge(
        (
            100
            * df["made_purchase_on_future_visit"]
            .value_counts(normalize=True)
            .rename("frac_made_purchase_on_future_visit")
        ).to_frame(),
        left_index=True,
        right_index=True,
    )
    .reset_index()
    .rename(columns={"index": "made_return_purchase"})
)

Query execution start time = 2023-04-13 17:03:04.444...done at 2023-04-13 17:03:17.048 (12.604 seconds).
Query returned 137,727 rows


,made_purchase_on_future_visit,num_return_purchasers_using_countif,num_return_purchasers_using_if_sum_casewhen,frac_made_purchase_on_future_visit
0,False,131734,131734,95.648638
1,True,5993,5993,4.351362


::: {.callout-tip title="Observations"}

1. `COUNTIF()` is a BigQuery SQL function ([1](https://cloud.google.com/bigquery/docs/reference/standard-sql/functions-and-operators#countif)) but it gives the same output as a standard SQL-based approach using `IF(SUM(CASE WHEN...))`. For the rest of this step, `COUNTIF()` will be used
2. Over the selected months, the class imbalance is close to 96% to 4% (or 96:4)
   - this comes from the `made_purchase_on_future_visit` column
   - these are the class labels for ML experiments
:::

### First Visit Attributes

**Question 2. Extract attributes from the first visit by visitors that made a purchase on a future visit.**

The following are the attributes extracted from the first visit (for the above visitors only) and the high-level categories that they belong to

1. geospatial and temporal
   - country
   - `datetime` attributes (day of month, hour of day, etc.)
2. metadata of each visit and visitor
   - these are *id* (or equivalent) columns
3. traffic sources and channels
   - traffic sources
     - these are search engines, social media networks, and other sources that result in visitors reaching the merchandise store's website ([link](https://support.google.com/analytics/answer/6205762?hl=en#understanding&zippy=%2Cin-this-article))
   - channels
     - these are groups of traffic sources ([link](https://support.google.com/analytics/answer/6010097?hl=en#zippy=%2Cin-this-article))
4. visitor activity on site
   - hits
   - bounces
   - page views
   - time spent on site
   - number of add-to-cart actions performed
5. visitor's device used to access site
   - browser
   - device category
   - operating system
6. label for machine learning
   - `made_purchase_on_future_visit`
     - same as in the above query
     - indicates whether a visitor makes a purchase during their next visit
7. Product
   - products viewed
   - products clicked
8. Promotion
   - promotions viewed (impressions)
   - promotions clicked

In [12]:
# | code-fold: false
query = f"""
        WITH
        -- Step 1. get visitors with a purchase on a future visit
        next_visit_purchasers AS (
             SELECT fullvisitorid,
                    IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, True, False) AS made_purchase_on_future_visit
             FROM `data-to-insights.ecommerce.web_analytics`
             WHERE date BETWEEN '{train_start_date}' AND '{test_end_date}'
             AND geoNetwork.country = 'United States'
             GROUP BY fullvisitorid
        ),
        -- Steps 2. and 3. get attributes of the first visit
        first_visit_attributes AS (
            SELECT -- =========== GEOSPATIAL AND TEMPORAL ATTRIBUTES OF VISIT ===========
                   geoNetwork.country,
                   EXTRACT(QUARTER FROM DATETIME(TIMESTAMP(TIMESTAMP_SECONDS(visitStartTime)), 'US/Pacific')) AS quarter,
                   EXTRACT(MONTH FROM DATETIME(TIMESTAMP(TIMESTAMP_SECONDS(visitStartTime)), 'US/Pacific')) AS month,
                   EXTRACT(DAY FROM DATETIME(TIMESTAMP(TIMESTAMP_SECONDS(visitStartTime)), 'US/Pacific')) AS day_of_month,
                   EXTRACT(DAYOFWEEK FROM DATETIME(TIMESTAMP(TIMESTAMP_SECONDS(visitStartTime)), 'US/Pacific')) AS day_of_week,
                   EXTRACT(HOUR FROM DATETIME(TIMESTAMP(TIMESTAMP_SECONDS(visitStartTime)), 'US/Pacific')) AS hour,
                   EXTRACT(MINUTE FROM DATETIME(TIMESTAMP(TIMESTAMP_SECONDS(visitStartTime)), 'US/Pacific')) AS minute,
                   EXTRACT(SECOND FROM DATETIME(TIMESTAMP(TIMESTAMP_SECONDS(visitStartTime)), 'US/Pacific')) AS second,
                   -- =========== VISIT AND VISITOR METADATA ===========
                   fullvisitorid,
                   visitId,
                   visitNumber,
                   DATETIME(TIMESTAMP(TIMESTAMP_SECONDS(visitStartTime)), 'US/Pacific') AS visitStartTime,
                   -- =========== SOURCE OF SITE TRAFFIC ===========
                   -- source of the traffic from which the visit was initiated
                   trafficSource.source,
                   -- medium of the traffic from which the visit was initiated
                   trafficSource.medium,
                   -- referring channel connected to visit
                   channelGrouping,
                   -- =========== VISITOR ACTIVITY ===========
                   -- total number of hits
                   (CASE WHEN totals.hits > 0 THEN totals.hits ELSE 0 END) AS hits,
                   -- number of bounces
                   (CASE WHEN totals.bounces > 0 THEN totals.bounces ELSE 0 END) AS bounces,
                   -- action performed during first visit
                   CAST(h.eCommerceAction.action_type AS INT64) AS action_type,
                   -- page views
                   IFNULL(totals.pageviews, 0) AS pageviews,
                   -- total revenue
                   totals.totalTransactionRevenue / 1000000 AS transact_revenue,
                   -- time on the website
                   IFNULL(totals.timeOnSite, 0) AS time_on_site,
                   -- whether add-to-cart was performed during visit
                   (CASE WHEN CAST(h.eCommerceAction.action_type AS INT64) = 3 THEN 1 ELSE 0 END) AS added_to_cart,
                   (CASE WHEN CAST(h.eCommerceAction.action_type AS INT64) = 2 THEN 1 ELSE 0 END) AS product_details_viewed,
                   -- =========== VISITOR DEVICES ===========
                   -- user's browser
                   device.browser,
                   -- user's operating system
                   device.operatingSystem AS os,
                   -- user's type of device
                   device.deviceCategory,
                   -- =========== PROMOTION ===========
                   h.promotion,
                   h.promotionActionInfo AS pa_info,
                   -- =========== PRODUCT ===========
                   h.product,
                   -- =========== ML LABEL (DEPENDENT VARIABLE) ===========
                   made_purchase_on_future_visit
            FROM `data-to-insights.ecommerce.web_analytics`,
            UNNEST(hits) AS h
            INNER JOIN next_visit_purchasers USING (fullvisitorid)
            WHERE date BETWEEN '{train_start_date}' AND '{train_end_date}'
            AND geoNetwork.country = 'United States'
            AND totals.newVisits = 1
        ),
        -- Step 4. get aggregated features (attributes) per visit
        visit_attributes AS (
            SELECT fullvisitorid,
                   visitId,
                   visitNumber,
                   visitStartTime,
                   country,
                   quarter,
                   month,
                   day_of_month,
                   day_of_week,
                   hour,
                   minute,
                   second,
                   source,
                   medium,
                   channelGrouping,
                   hits,
                   bounces,
                   -- get the last action performed during the first visit
                   -- (this indicates where the visitor left off at the end of their visit)
                   MAX(action_type) AS last_action,
                   -- get number of products whose details were viewed
                   SUM(product_details_viewed) AS product_detail_views,
                   -- get number of promotions displayed and clicked during the first visit
                   COUNT(CASE WHEN pa_info IS NOT NULL THEN pa_info.promoIsView ELSE NULL END) AS promos_displayed,
                   COUNT(CASE WHEN pa_info IS NOT NULL THEN pa_info.promoIsClick ELSE NULL END) AS promos_clicked,
                   -- get number of products displayed and clicked during the first visit
                   COUNT(CASE WHEN pu.isImpression IS NULL THEN NULL ELSE 1 END) AS product_views,
                   COUNT(CASE WHEN pu.isClick IS NULL THEN NULL ELSE 1 END) AS product_clicks,
                   pageviews,
                   transact_revenue,
                   time_on_site,
                   browser,
                   os,
                   deviceCategory,
                   SUM(added_to_cart) AS added_to_cart,
                   made_purchase_on_future_visit,
            FROM first_visit_attributes
            LEFT JOIN UNNEST(promotion) as p
            LEFT JOIN UNNEST(product) as pu
            GROUP BY fullvisitorid,
                     visitId,
                     visitNumber,
                     visitStartTime,
                     country,
                     quarter,
                     month,
                     day_of_month,
                     day_of_week,
                     hour,
                     minute,
                     second,
                     source,
                     medium,
                     channelGrouping,
                     hits,
                     bounces,
                     pageviews,
                     transact_revenue,
                     time_on_site,
                     browser,
                     os,
                     deviceCategory,
                     made_purchase_on_future_visit
        )
        SELECT *
        FROM visit_attributes
        """
df = run_sql_query(query, **gcp_auth_dict, show_df=False)
with pd.option_context("display.max_columns", None):
    display(df.head())
    display(df.tail())

Query execution start time = 2023-04-13 17:03:44.045...done at 2023-04-13 17:04:06.815 (22.770 seconds).
Query returned 92,859 rows


,fullvisitorid,visitId,visitNumber,visitStartTime,country,quarter,month,day_of_month,day_of_week,hour,minute,second,source,medium,channelGrouping,hits,bounces,last_action,product_detail_views,promos_displayed,promos_clicked,product_views,product_clicks,pageviews,transact_revenue,time_on_site,browser,os,deviceCategory,added_to_cart,made_purchase_on_future_visit
0,8163735676529750721,1481176805,1,2016-12-07 22:00:05,United States,4,12,7,4,22,0,5,google,cpc,Paid Search,5,0,0,0,0,0,0,0,5,NaN,109,Chrome,Windows,desktop,0,False
1,0897634596694862660,1475199828,1,2016-09-29 18:43:48,United States,3,9,29,5,18,43,48,google,organic,Organic Search,21,0,2,2,9,0,90,2,19,NaN,467,Chrome,Windows,desktop,0,False
2,3891893273235584028,1478844153,1,2016-11-10 22:02:33,United States,4,11,10,5,22,2,33,youtube.com,referral,Social,11,0,2,1,18,0,36,1,10,NaN,1003,Safari (in-app),iOS,tablet,0,False
3,6952706734234495703,1480581075,1,2016-12-01 00:31:15,United States,4,12,1,5,0,31,15,google,organic,Organic Search,9,0,0,0,27,1,24,0,8,NaN,141,Chrome,Android,mobile,0,False
4,3008949133172215443,1475213353,1,2016-09-29 22:29:13,United States,3,9,29,5,22,29,13,google,organic,Organic Search,14,0,2,3,9,0,24,3,11,NaN,229,Chrome,Macintosh,desktop,0,False


,fullvisitorid,visitId,visitNumber,visitStartTime,country,quarter,month,day_of_month,day_of_week,hour,minute,second,source,medium,channelGrouping,hits,bounces,last_action,product_detail_views,promos_displayed,promos_clicked,product_views,product_clicks,pageviews,transact_revenue,time_on_site,browser,os,deviceCategory,added_to_cart,made_purchase_on_future_visit
92854,1446285229092173706,1478190491,1,2016-11-03 09:28:11,United States,4,11,3,5,9,28,11,(direct),(none),Direct,3,0,0,0,0,0,29,0,3,NaN,224,Chrome,Chrome OS,desktop,0,True
92855,927804845306410814,1482181118,1,2016-12-19 12:58:38,United States,4,12,19,2,12,58,38,google,organic,Organic Search,3,0,0,0,0,0,32,0,3,NaN,83,Chrome,Linux,desktop,0,False
92856,8969674851394509099,1482941116,1,2016-12-28 08:05:16,United States,4,12,28,4,8,5,16,youtube.com,referral,Social,3,0,0,0,0,0,28,0,3,NaN,85,Chrome,Windows,desktop,0,False
92857,0486118040507415508,1477164641,1,2016-10-22 12:30:41,United States,4,10,22,7,12,30,41,t.co,referral,Social,3,0,0,0,0,0,28,0,3,NaN,78,Chrome,Windows,desktop,0,False
92858,9088496055778533168,1482668980,1,2016-12-25 04:29:40,United States,4,12,25,1,4,29,40,youtube.com,referral,Social,3,0,0,0,0,0,36,0,3,NaN,69,Chrome,Linux,desktop,0,False


::: {.callout-note title="Notes"}

1. Below is a brief overview of the CTEs used here
   - `next_visit_purchasers`
     - gets visitors who made a purchase on a return visit to the merchandise store on the Google Marketplace
   - `first_visit_attributes`
     - gets attributes of first visit
     - the statement `INNER JOIN next_visit_purchasers USING (fullvisitorid)` is used to only select the visitors that made a purchase on a return visit to the store (these `fullvisitorid`s are stored in the `next_visit_purchasers` CTE)
   - `visit_attributes`
     - aggregates values from nested columns to get views and clicks for promotions and products
2. The BigQuery SQL function `UNNEST` was used to flatten nested columns.
3. The start and end dates of the ML training, validation and test data splits were defined. The training dates have been used to filter the `first_visit_attributes` CTE in order since EDA in this step will only be performed using the training data in order to avoid data leakage (or lookahead bias).
4. The SQL required to extract most of these columns was determined from (a) the documentation for the dataset and (b) examining the first few rows of the dataset in these columns. For brevity, we won't discuss these columns in further detail. These column categories are listed below
   - geospatial and temporal
   - metadata of each visit and visitor
   - traffic sources and channels
   - visitor activity on merchandise store website
   - visitor's device
   - label for machine learning (discussed in *Data Preparation Question 2. above*)
5. These columns were extracted based on intuition about the attributes of each visit that will help to predict the probability of a visitor making a purchase on a future (return) visit to the merchandise store.
:::

### Fraction of Visitors with Add-to-Cart on First Visit

**Question 3. What fraction of visitors added one or more items to their shopping cart during their first visit?**

In [13]:
# | code-fold: false
for c in ["added_to_cart"]:
    display(
        (100 * df[c].value_counts(dropna=False, normalize=True))
        .rename("number")
        .reset_index()
        .rename(columns={"index": "added_to_cart"})
    )

,added_to_cart,number
0,0,89.336521
1,1,6.335412
2,2,2.073035
3,3,0.915366
4,4,0.479221
5,5,0.273533
6,6,0.176612
7,7,0.102306
8,8,0.078614
9,9,0.041999


::: {.callout-tip title="Observations"}

1. During the months covered by the training data, nearly 90% of visitors did not add an item to their shopping cart during their first visit to the merchandise store (`added_to_cart = 0`). Only 10% of such visitors added an item to their shopping cart during this time.
:::

### Reasons For and Handling of Duplicates

**Question 4. Comment on duplicates present in the data prepared above. What are some possible reasons for the presence of duplicates in the above prepared data? How should these be handled?**

Below we show that there are duplicates within the `fullvisitorid` column

In [14]:
print(
    f"Number of rows = {len(df):,}\nNumber of unique visitor IDs = "
    f"{df['fullvisitorid'].nunique():,}\n"
    f"Largest visitNumber = {df['visitNumber'].max()}"
)

Number of rows = 92,859
Number of unique visitor IDs = 92,551
Largest visitNumber = 1


These duplicates are retrieved below, showing that multiple `visitId`s are present for the same `visitorid`

In [15]:
# | code-fold: false
df_num_dups = (
    df.groupby(["fullvisitorid"])
    .agg({"visitId": "count", "visitNumber": "max"})
    .reset_index()
    .rename(columns={"visitId": "num_visitIds"})
    .query("num_visitIds > 1")
)
display(df_num_dups.head())
display(df_num_dups.tail())

,fullvisitorid,num_visitIds,visitNumber
124,0014997413479849928,2,1
1186,012569301201854368,2,1
1400,0153393931967124172,2,1
1831,0196238382136996118,2,1
2163,0233922069260074966,2,1


,fullvisitorid,num_visitIds,visitNumber
91561,9897914422695841426,2,1
91653,9906208132011345120,2,1
91749,9915457192772678365,2,1
92054,9949751653823311987,2,1
92079,9952616174324085427,2,1


::: {.callout-tip title="Observations"}

1. Duplicates can occur by
   - `fullvisitorid`
   - `visitId`

   so, we should explore both cases separately.
:::

Duplicated `visitId`s are shown below

In [16]:
# | code-fold: false
dup_visit_ids = df[df.duplicated(subset=["visitId"], keep=False)]
num_dups = len(df[df.duplicated(subset=["visitId"], keep="first")])
print(
    f"Found {num_dups:,} duplicated visitIds out of "
    f"{len(df):,} ({100*num_dups/len(df):.3f}%)"
)
with pd.option_context("display.max_columns", None):
    display(dup_visit_ids.sort_values(by=["visitId"]).head(25))

Found 742 duplicated visitIds out of 92,859 (0.799%)


,fullvisitorid,visitId,visitNumber,visitStartTime,country,quarter,month,day_of_month,day_of_week,hour,minute,second,source,medium,channelGrouping,hits,bounces,last_action,product_detail_views,promos_displayed,promos_clicked,product_views,product_clicks,pageviews,transact_revenue,time_on_site,browser,os,deviceCategory,added_to_cart,made_purchase_on_future_visit
28174,7673928089571501866,1472751431,1,2016-09-01 10:37:11,United States,3,9,1,5,10,37,11,google,organic,Organic Search,27,0,2,6,0,0,293,8,19,NaN,845,Chrome,Windows,desktop,0,False
47204,6028120763017470092,1472751431,1,2016-09-01 10:37:11,United States,3,9,1,5,10,37,11,dfa,cpm,Display,7,0,0,0,9,0,147,0,7,NaN,153,Chrome,Android,mobile,0,False
66489,4654710059786315542,1472752926,1,2016-09-01 11:02:06,United States,3,9,1,5,11,2,6,youtube.com,referral,Social,3,0,0,0,18,0,0,0,3,NaN,50,Chrome,Windows,desktop,0,False
14790,0622219713224273207,1472752926,1,2016-09-01 11:02:06,United States,3,9,1,5,11,2,6,mall.googleplex.com,referral,Referral,7,0,0,0,9,0,167,0,7,NaN,178,Chrome,Linux,desktop,0,False
79228,0983194581450463928,1472771118,1,2016-09-01 16:05:18,United States,3,9,1,5,16,5,18,reddit.com,referral,Social,2,0,0,0,9,0,34,0,2,NaN,15,Chrome,Windows,desktop,0,False
64049,8679833862264857329,1472771118,1,2016-09-01 16:05:18,United States,3,9,1,5,16,5,18,google,organic,Organic Search,1,1,0,0,0,0,0,0,1,NaN,0,Safari,Macintosh,desktop,0,False
62058,5501176514964856126,1472799309,1,2016-09-01 23:55:09,United States,3,9,1,5,23,55,9,google,organic,Organic Search,12,0,2,4,0,0,108,4,8,NaN,288,Chrome,Chrome OS,desktop,0,False
83797,5501176514964856126,1472799309,1,2016-09-02 00:00:01,United States,3,9,2,6,0,0,1,google,organic,Organic Search,5,0,0,0,0,0,143,0,5,NaN,75,Chrome,Chrome OS,desktop,0,False
5052,3718403740052363161,1472826491,1,2016-09-02 07:28:11,United States,3,9,2,6,7,28,11,google,organic,Organic Search,27,0,2,9,36,2,70,9,16,NaN,196,Safari,iOS,tablet,0,False
25190,0841953213802800334,1472826491,1,2016-09-02 07:28:11,United States,3,9,2,6,7,28,11,(direct),(none),Direct,83,0,3,15,45,0,361,20,59,NaN,873,Chrome,Macintosh,desktop,4,False


::: {.callout-tip title="Observations"}

1. For the same `visitId`, different traffic sources (`source`, `medium`, `channelGrouping`) bring the same or different visitors (`fullvisitorid`) to the website at the same `datetime` (`visitStartTime`). Google Analytics assigns the same `visitId` to such visitors. There are two types of nested duplicates here
   - the same visitor accessing the merchandise store from
     - multiple devices at the same time
       - this cross-device tracking appears to be [allowed by Google Analytics](https://blog.google/products/marketingplatform/360/cross-device-capabilities/)
     - the same device and same browser (using separate browser windows after clearing cookies) at the same time
       - this is also [allowed by Google Analytics](https://www.quora.com/In-Google-Analytics-why-do-I-have-more-unique-visitors-than-visits)
   - different visitors are accessing the merchandise store from multiple devices at the same time
     - this is not a duplicated occurrence
     - most likely this corresponds to two distinct visitors who happened to navigate to the site at the same time
2. There are a negligible number of such duplicates in the dataset.
:::

Duplicated `fullvisitorId`s are shown below

In [17]:
# | code-fold: false
dup_visitor_ids = df[df.duplicated(subset=["fullvisitorid"], keep=False)]
num_dups = len(df[df.duplicated(subset=["fullvisitorid"], keep="first")])
print(
    f"Found {num_dups:,} duplicated fullvisitorid out of "
    f"{len(df):,} ({100*num_dups/len(df):.3f}%)"
)
with pd.option_context("display.max_columns", None):
    display(dup_visitor_ids.sort_values(by=["fullvisitorid"]).head(25))

Found 308 duplicated fullvisitorid out of 92,859 (0.332%)


,fullvisitorid,visitId,visitNumber,visitStartTime,country,quarter,month,day_of_month,day_of_week,hour,minute,second,source,medium,channelGrouping,hits,bounces,last_action,product_detail_views,promos_displayed,promos_clicked,product_views,product_clicks,pageviews,transact_revenue,time_on_site,browser,os,deviceCategory,added_to_cart,made_purchase_on_future_visit
50597,0014997413479849928,1477513747,1,2016-10-26 13:29:07,United States,4,10,26,4,13,29,7,(direct),(none),Direct,14,0,2,1,18,0,38,4,10,NaN,77,Chrome,Macintosh,desktop,0,False
62473,0014997413479849928,1474324872,1,2016-09-19 15:41:12,United States,3,9,19,2,15,41,12,mall.googleplex.com,referral,Referral,17,0,2,3,9,0,168,5,12,NaN,349,Chrome,Macintosh,desktop,0,False
6720,012569301201854368,1477496472,1,2016-10-26 08:41:12,United States,4,10,26,4,8,41,12,analytics.google.com,referral,Referral,1,1,0,0,9,0,0,0,1,NaN,0,Chrome,Macintosh,desktop,0,False
32410,012569301201854368,1477339547,1,2016-10-24 13:05:52,United States,4,10,24,2,13,5,52,(direct),(none),Direct,2,0,0,0,9,0,10,0,2,NaN,12,Chrome,Macintosh,desktop,0,False
11900,0153393931967124172,1481528488,1,2016-12-11 23:41:28,United States,4,12,11,1,23,41,28,google,organic,Organic Search,5,0,0,0,9,0,15,0,5,NaN,1105,Safari,Macintosh,desktop,0,False
62110,0153393931967124172,1481528488,1,2016-12-12 00:00:03,United States,4,12,12,2,0,0,3,google,organic,Organic Search,13,0,2,1,9,0,92,2,11,NaN,614,Safari,Macintosh,desktop,0,False
11590,0196238382136996118,1482220536,1,2016-12-20 00:00:19,United States,4,12,20,3,0,0,19,(direct),(none),Direct,4,0,6,0,0,0,0,0,4,165.0,61,Chrome,Macintosh,desktop,0,False
15516,0196238382136996118,1482220536,1,2016-12-19 23:55:36,United States,4,12,19,2,23,55,36,(direct),(none),Direct,11,0,5,1,0,0,7,1,9,NaN,99,Chrome,Macintosh,desktop,1,False
72238,0233922069260074966,1474926165,1,2016-09-26 14:42:45,United States,3,9,26,2,14,42,45,(direct),(none),Direct,5,0,0,0,0,0,18,0,5,NaN,869,Chrome,Macintosh,desktop,0,False
91048,0233922069260074966,1477516633,1,2016-10-26 14:17:13,United States,4,10,26,4,14,17,13,mall.googleplex.com,referral,Referral,3,0,0,0,9,0,14,0,3,NaN,46,Chrome,Macintosh,desktop,0,False


::: {.callout-tip title="Observations"}

1. For the same `fullvisitorid`, different traffic sources (`source`, `medium`, `channelGrouping`) bring the same visitor (`fullvisitorid`) to the website at the different `datetime`s (`visitStartTime`s) from the same device (`browser`, `os`, `deviceCategory`). There are two types of nested duplicates here
   - the same visit by the same visitor with a <30 minute period of inactivity between duplicates
     - (by default) [Google Analytics allows up to 30 minutes of inactivity before starting a new visit](https://support.google.com/analytics/answer/2731565?hl=en#time-based-expiration&zippy=%2Cin-this-article), so it is not clear why such a short period of inactivity should start a new *instance* of the same visit instead of just accumulating stats into the same *instance*
   - a different visit by the same visitor with a >30 minute period of inactivity between duplicates
     - it is also not clear why these duplicates are present in the data
2. Since
   - the use-case for this project requires attributes of **only** the first visit (per visitor) to be used to predict the probability of a purchase during a future visit by the same visitor
   - it is not clear why this type of duplicated record is present in the prepared data

   we will want to drop this type of duplicate from the training, validation and test splits of the prepared data (we're assuming that the same type of problem can occur throughout the dataset and not just in the training data).
3. There are a negligible number of such duplicates in the dataset.
:::

With this in mind, columns with duplicates in the `fullvisitorid` column are dropped. This will be done using Python

In [18]:
# | code-fold: false
df = df.drop_duplicates(subset=["fullvisitorid"], keep="first")

### Nested Promotion Column

**Question 5. Show and comment on unique values in the nested promotion column.**

The number of promotions and products displayed (impressions) and clicked are shown below

In [19]:
# | code-fold: false
for c in [
    "promos_displayed",
    "promos_clicked",
    "product_views",
    "product_clicks",
    "product_detail_views",
]:
    df_num_visitor_counts = (
        df[c]
        .value_counts(dropna=False)
        .rename("num_visitors")
        .reset_index()
        .rename(columns={"index": f"num_{c}"})
    )
    assert (
        type(df_num_visitor_counts.query("num_visitors == 0")[c].squeeze()).__name__
        == "NAType"
    )
    display(df_num_visitor_counts.query("num_visitors > 0"))

,promos_displayed,num_visitors
0,9,42840
1,0,28655
2,18,12938
3,27,4079
4,36,1729
5,45,875
6,54,426
7,13,251
8,63,242
9,72,144


,promos_clicked,num_visitors
0,0,78328
1,1,10802
2,2,2019
3,3,752
4,4,318
5,5,145
6,6,76
7,7,47
8,8,20
9,9,13


,product_views,num_visitors
0,0,26848
1,12,14273
2,24,4663
3,36,2343
4,7,2271
...,...,...
602,366,1
603,685,1
604,647,1
605,719,1


,product_clicks,num_visitors
0,0,68175
1,1,8680
2,2,5455
3,3,3056
4,4,2036
...,...,...
62,42,1
63,125,1
64,71,1
65,48,1


,product_detail_views,num_visitors
0,0,68220
1,1,10547
2,2,5197
3,3,2928
4,4,1734
...,...,...
56,41,1
57,82,1
58,31,1
59,122,1


::: {.callout-note title="Notes"}

1. Product views are the number of times a product was seen while in a list of other products (eg. on a product listing page or in a product category page).
2. Product clicks are the number of times a product was clicked on after being viewed.
3. Product detail views are the number of times a visitor has visited a product's page (not just viewed its details as part of a product listing).
   - a visitor might have viewed product details page for products that are
     - part of a product listing
     - not part of a product listing
4. Similar logic applies to promotions (eg. banners) views and clicks.
:::

::: {.callout-tip title="Observations"}

1. Products and promotions on the merchandise store's website on the Google Marketplace are not being
   - viewed (as part of a listing or in detail)
   - clicked

   often.
:::

Promotion-related columns are flattened and shown (see `promotionActionInfo`) for a single visit

In [20]:
# | code-fold: false
query = f"""
        WITH visit_promotion_attrs AS (
            SELECT fullvisitorid,
                   visitId,
                   visitNumber,
                   DATETIME(TIMESTAMP(TIMESTAMP_SECONDS(visitStartTime)), 'US/Pacific') AS visitStartTime,
                   CAST(h.ecommerceaction.action_type AS INT64) AS action_type,
                   h.promotion,
                   h.promotionActionInfo AS pa_info,
                   trafficSource.source,
                   trafficSource.medium,
                   channelGrouping,
                   device.browser,
                   device.operatingSystem,
                   device.deviceCategory
            FROM `data-to-insights.ecommerce.web_analytics`,
            UNNEST(hits) AS h
            WHERE visitId = 1476880065  -- 1476880065, 1478579523, 1474972357, 1478844153
            AND geoNetwork.country = 'United States'
        )
        SELECT * EXCEPT(promotion, promoId, pa_info, visitStartTime),
               pa_info,
               CASE WHEN pa_info IS NOT NULL THEN pa_info.promoIsView ELSE NULL END AS view_promo,
               CASE WHEN pa_info IS NOT NULL THEN pa_info.promoIsClick ELSE NULL END AS click_promo
        FROM visit_promotion_attrs
        LEFT JOIN UNNEST(promotion) as p
        """
df_raw = run_sql_query(query, **gcp_auth_dict, show_df=False)
df_raw["action_type"] = df_raw["action_type"].map(mapper)
with pd.option_context("display.max_colwidth", None, "display.max_rows", None):
    display(df_raw.head(10))

Query execution start time = 2023-04-13 17:07:00.571...done at 2023-04-13 17:07:02.254 (1.683 seconds).
Query returned 42 rows


,fullvisitorid,visitId,visitNumber,action_type,source,medium,channelGrouping,browser,operatingSystem,deviceCategory,promoName,promoCreative,promoPosition,pa_info,view_promo,click_promo
0,3072592563711482446,1476880065,1,Unknown,google,organic,Organic Search,Chrome,Android,mobile,None,None,None,None,<NA>,<NA>
1,3072592563711482446,1476880065,1,Unknown,google,organic,Organic Search,Chrome,Android,mobile,None,None,None,None,<NA>,<NA>
2,3072592563711482446,1476880065,1,Unknown,google,organic,Organic Search,Chrome,Android,mobile,None,None,None,None,<NA>,<NA>
3,3072592563711482446,1476880065,1,Unknown,google,organic,Organic Search,Chrome,Android,mobile,None,None,None,None,<NA>,<NA>
4,3072592563711482446,1476880065,1,Unknown,google,organic,Organic Search,Chrome,Android,mobile,Apparel,home_main_link_apparel.jpg,Row 1,"{'promoIsView': True, 'promoIsClick': None}",True,<NA>
5,3072592563711482446,1476880065,1,Unknown,google,organic,Organic Search,Chrome,Android,mobile,Backpacks,home_bags_google_2.jpg,Row 2 Combo,"{'promoIsView': True, 'promoIsClick': None}",True,<NA>
6,3072592563711482446,1476880065,1,Unknown,google,organic,Organic Search,Chrome,Android,mobile,Mens T-Shirts,mens-tshirts.jpg,Row 3-1,"{'promoIsView': True, 'promoIsClick': None}",True,<NA>
7,3072592563711482446,1476880065,1,Unknown,google,organic,Organic Search,Chrome,Android,mobile,Womens T-Shirts,womens-tshirts.jpg,Row 3-2,"{'promoIsView': True, 'promoIsClick': None}",True,<NA>
8,3072592563711482446,1476880065,1,Unknown,google,organic,Organic Search,Chrome,Android,mobile,Office,green_row_link_to_office.jpg,Row 5 Color Combo,"{'promoIsView': True, 'promoIsClick': None}",True,<NA>
9,3072592563711482446,1476880065,1,Unknown,google,organic,Organic Search,Chrome,Android,mobile,Drinkware,red_row_hydrate.jpg,Row 4 Color Combo,"{'promoIsView': True, 'promoIsClick': None}",True,<NA>


::: {.callout-note title="Notes"}

1. `promotionActionInfo` contains information about visitor views and clicks.
2. The `CASE WHEN` was constructed for both `view_promo` and `click_promo` columns based on the nested `promotionActionInfo` (mapped to `pa_info`) column.
:::

::: {.callout-tip title="Observations"}

1. When `view_promo = True`, a visitor has viewed a promotion. If it is not viewed, then it is `NULL`.
2. When a visitor clicks a promotion after viewing it
   - `click_promo = True`
   - `view_promo` is `NULL`
     - this prevents double-counting a promotion that is both viewed and clicked
:::

### Nested Product Column

**Question 6. Show and comment on unique values in the nested product column.**

Product-related columns are flattened and shown for a single visit

In [21]:
# | code-fold: false
query = f"""
        WITH visit_product_attrs AS (
            SELECT fullvisitorid,
               visitId,
               visitNumber,
               DATETIME(TIMESTAMP(TIMESTAMP_SECONDS(visitStartTime)), 'US/Pacific') AS visitStartTime,
               CAST(h.ecommerceaction.action_type AS INT64) AS action_type,
               h.product,
               (CASE WHEN ARRAY_LENGTH(h.product) = 0 THEN 0 ELSE ARRAY_LENGTH(h.product) END) AS product_count,
               (CASE WHEN CAST(h.eCommerceAction.action_type AS INT64) = 2 THEN 1 ELSE 0 END) AS product_details_viewed,
               trafficSource.source,
               trafficSource.medium,
               channelGrouping,
               device.browser,
               device.operatingSystem,
               device.deviceCategory
            FROM `data-to-insights.ecommerce.web_analytics`,
            UNNEST(hits) AS h
            WHERE visitId = 1478579523  -- 1478579523, 1474972357
            AND geoNetwork.country = 'United States'
        )
        SELECT *,
               p.isImpression AS viewed_product,
               p.isClick AS clicked_product
        FROM visit_product_attrs
        LEFT JOIN UNNEST(product) as p
        """
df_raw = run_sql_query(query, **gcp_auth_dict, show_df=False)
df_raw["action_type"] = df_raw["action_type"].map(mapper)
with pd.option_context("display.max_columns", None):
    display(df_raw.head())
    display(df_raw.tail())

Query execution start time = 2023-04-13 17:07:08.966...done at 2023-04-13 17:07:11.872 (2.906 seconds).
Query returned 266 rows


,fullvisitorid,visitId,visitNumber,visitStartTime,action_type,product,product_count,product_details_viewed,source,medium,channelGrouping,browser,operatingSystem,deviceCategory,productSKU,v2ProductName,v2ProductCategory,productVariant,productBrand,productRevenue,localProductRevenue,productPrice,localProductPrice,productQuantity,productRefundAmount,localProductRefundAmount,isImpression,isClick,customDimensions,customMetrics,productListName,productListPosition,viewed_product,clicked_product
0,7270403007208566857,1478579523,1,2016-11-07 20:32:03,Unknown,[],0,0,siliconvalley.about.com,referral,Referral,Chrome,Chrome OS,desktop,None,None,None,None,None,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],[],None,<NA>,<NA>,<NA>
1,7270403007208566857,1478579523,1,2016-11-07 20:32:03,Unknown,[],0,0,siliconvalley.about.com,referral,Referral,Chrome,Chrome OS,desktop,None,None,None,None,None,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],[],None,<NA>,<NA>,<NA>
2,7270403007208566857,1478579523,1,2016-11-07 20:32:03,Unknown,[],0,0,siliconvalley.about.com,referral,Referral,Chrome,Chrome OS,desktop,None,None,None,None,None,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],[],None,<NA>,<NA>,<NA>
3,7270403007208566857,1478579523,1,2016-11-07 20:32:03,Unknown,[],0,0,siliconvalley.about.com,referral,Referral,Chrome,Chrome OS,desktop,None,None,None,None,None,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],[],None,<NA>,<NA>,<NA>
4,7270403007208566857,1478579523,1,2016-11-07 20:32:03,Unknown,[],0,0,siliconvalley.about.com,referral,Referral,Chrome,Chrome OS,desktop,None,None,None,None,None,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],[],None,<NA>,<NA>,<NA>


,fullvisitorid,visitId,visitNumber,visitStartTime,action_type,product,product_count,product_details_viewed,source,medium,channelGrouping,browser,operatingSystem,deviceCategory,productSKU,v2ProductName,v2ProductCategory,productVariant,productBrand,productRevenue,localProductRevenue,productPrice,localProductPrice,productQuantity,productRefundAmount,localProductRefundAmount,isImpression,isClick,customDimensions,customMetrics,productListName,productListPosition,viewed_product,clicked_product
261,7270403007208566857,1478579523,1,2016-11-07 20:32:03,Unknown,"[{'productSKU': 'GGOEGAAX0318', 'v2ProductName...",12,0,siliconvalley.about.com,referral,Referral,Chrome,Chrome OS,desktop,GGOEGAAX0686,YouTube Youth Short Sleeve Tee Red,Home/Shop by Brand/YouTube/,(not set),(not set),<NA>,<NA>,18990000,18990000,<NA>,<NA>,<NA>,True,<NA>,[],[],Category,9,True,<NA>
262,7270403007208566857,1478579523,1,2016-11-07 20:32:03,Unknown,"[{'productSKU': 'GGOEGAAX0318', 'v2ProductName...",12,0,siliconvalley.about.com,referral,Referral,Chrome,Chrome OS,desktop,GGOEYHPA003510,YouTube Trucker Hat,Home/Shop by Brand/YouTube/,(not set),(not set),<NA>,<NA>,21990000,21990000,<NA>,<NA>,<NA>,True,<NA>,[],[],Category,10,True,<NA>
263,7270403007208566857,1478579523,1,2016-11-07 20:32:03,Unknown,"[{'productSKU': 'GGOEGAAX0318', 'v2ProductName...",12,0,siliconvalley.about.com,referral,Referral,Chrome,Chrome OS,desktop,GGOEGAAX0330,YouTube Men's Skater Tee Charcoal,Home/Shop by Brand/YouTube/,(not set),(not set),<NA>,<NA>,19990000,19990000,<NA>,<NA>,<NA>,True,<NA>,[],[],Category,11,True,<NA>
264,7270403007208566857,1478579523,1,2016-11-07 20:32:03,Unknown,"[{'productSKU': 'GGOEGAAX0318', 'v2ProductName...",12,0,siliconvalley.about.com,referral,Referral,Chrome,Chrome OS,desktop,GGOEYDHJ056099,22 oz YouTube Bottle Infuser,Home/Shop by Brand/YouTube/,(not set),(not set),<NA>,<NA>,4990000,4990000,<NA>,<NA>,<NA>,True,<NA>,[],[],Category,12,True,<NA>
265,7270403007208566857,1478579523,1,2016-11-07 20:32:03,Unknown,[],0,0,siliconvalley.about.com,referral,Referral,Chrome,Chrome OS,desktop,None,None,None,None,None,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],[],None,<NA>,<NA>,<NA>


::: {.callout-tip title="Observations"}

1. The `product` column is a nested column with a multi-element array. Each element (dictionary) of the array corresponds to a different product in a listing or category of products. With the BigQuery `UNNEST()` function, this column is exploded into the following standalone columns
   - `productSKU`
   - `v2ProductName`
   - `v2ProductCategory`
   - `productVariant`
   - `productBrand`
   - `productRevenue`
   - `localProductRevenue`
   - `productPrice`
   - `localProductPrice`
   - `productQuantity`
   - `productRefundAmount`
   - `localProductRefundAmount`
   - `isImpression`
   - `isClick`
   - `customDimensions`
   - `customMetrics`
   - `productListName`
   - `productListPosition`
   - `viewed_product`
   - `clicked_product`
2. `viewed_product` is `True` for every product in the product listing that was viewed.
:::

If a product is viewed in a listing (`product_count > 0`) during a visit, then there are only two possible values for `clicked_product` and `viewed_product`, as shown below

In [22]:
# | code-fold: false
for c in ["viewed_product", "clicked_product"]:
    # show unique values
    display(
        df_raw.query("product_count > 0")[c].value_counts(dropna=False).reset_index()
    )

    # verify that False is not a unique value
    assert df_raw.query("product_count > 0").query(f"{c} == False").empty

,viewed_product,count
0,True,189
1,<NA>,41


,clicked_product,count
0,<NA>,213
1,True,17


If a product is not viewed in a listing, then the only value in these same two columns is `NULL` since they come from a nested column `product` which contains an empty array `[]` if a product is such a scenario.

This is shown below

In [23]:
# | code-fold: false
for c in ["viewed_product", "clicked_product"]:
    display(
        df_raw.query("product_count == 0")[c].value_counts(dropna=False).reset_index()
    )

,viewed_product,count
0,<NA>,36


,clicked_product,count
0,<NA>,36


For every product in the product listing that was viewed and clicked

- `clicked_product` is `True`
- `viewed_product` is `NULL`

which prevents double-counting products that are both viewed and clicked (similar to for promotions), as shown below

In [24]:
display(df_raw.query("clicked_product == True")[["viewed_product", "clicked_product"]])

,viewed_product,clicked_product
20,<NA>,True
23,<NA>,True
42,<NA>,True
44,<NA>,True
75,<NA>,True
140,<NA>,True
149,<NA>,True
152,<NA>,True
155,<NA>,True
158,<NA>,True


Product detail views and clicking of products that were viewed in a product or product category listing can also be retrieved from the `action_type` column, which tracks each action performed by a visitor during a visit. Its unique values are shown below

In [25]:
df_raw["action_type"].value_counts(dropna=False).reset_index()

,action_type,count
0,Unknown,225
1,Product detail views,22
2,Click through of product lists,17
3,Add product(s) to cart,2


Below, we verify that the products that were clicked can be equivalently determined using separated nested columns

- `clicked_product` (extracted from nested column `product`)
  - `clicked_product == True`
- `action_type` (extracted from nested column `hits`)
  - `action_type == 'Click through of product lists'`

In [26]:
# | code-fold: false
visit_prodict_view_click_cols = [
    "fullvisitorid",
    "visitStartTime",
    "action_type",
    "product_details_viewed",
    "isImpression",
    "isClick",
    "viewed_product",
    "clicked_product",
]
assert df_raw.query("clicked_product == True")[visit_prodict_view_click_cols].equals(
    df_raw.query("action_type == 'Click through of product lists'")[
        visit_prodict_view_click_cols
    ]
)

When a product is viewed in a listing (`viewed_product`), during a visit, the product count for those visits is greater than zero

In [27]:
# | code-fold: false
assert df_raw.query("viewed_product == True")["product_count"].min() > 0
display(df_raw.query("viewed_product == True")["product_count"].describe().to_frame())

,product_count
count,189.0
mean,9.719577
std,3.355004
min,2.0
25%,6.0
50%,12.0
75%,12.0
max,12.0


For informational purposes, the raw dataset without unnesting the products and promotions columns is shown below for a small number of visits

In [28]:
# | code-fold: false
visit_ids_dict = {
    1478844153: "papayawhip",
    1476880065: "mistyrose",
    1478579523: "lavender",
    1474972357: "lightcyan",
}
visit_ids_str = "(" + ", ".join([str(v) for v in list(visit_ids_dict)]) + ")"

Attributes for these visits are retrieved below without unnesting `product` and `promotion`

In [29]:
# | code-fold: false
query = f"""
        WITH visit_promotion_attrs AS (
            SELECT fullvisitorid,
                   visitId,
                   visitNumber,
                   DATETIME(TIMESTAMP(TIMESTAMP_SECONDS(visitStartTime)), 'US/Pacific') AS visitStartTime,
                   CAST(h.ecommerceaction.action_type AS INT64) AS action_type,
                   (CASE WHEN CAST(h.eCommerceAction.action_type AS INT64) = 2 THEN 1 ELSE 0 END) AS product_details_viewed,
                   trafficSource.source,
                   trafficSource.medium,
                   channelGrouping,
                   device.browser,
                   device.operatingSystem,
                   device.deviceCategory,
                   -- visit
                   totals.timeOnSite,
                   totals.timeOnScreen,
                   totals.visits,
                   totals.totalTransactionRevenue / 1000000 AS transact_revenue,
                   -- nested columns
                   h.product,
                   h.promotion,
                   -- experimental columns that were not used
                   h.isInteraction,
                   trafficSource.campaign,
                   trafficSource.isTrueDirect,
            FROM `data-to-insights.ecommerce.web_analytics`,
            UNNEST(hits) AS h
            WHERE visitId IN {visit_ids_str}
        )
        SELECT * EXCEPT(visitStartTime)
        FROM visit_promotion_attrs
        """
df_raw = run_sql_query(query, **gcp_auth_dict, show_df=False)
df_raw["action_type"] = df_raw["action_type"].map(mapper)

Query execution start time = 2023-04-13 17:07:43.363...done at 2023-04-13 17:07:46.085 (2.722 seconds).
Query returned 143 rows


These attributes can be shown per `visitId` using

```python
for visit_id in list(visit_ids_dict):
    with pd.option_context("display.max_columns", None, "display.max_rows", None):
        display(df_raw.query(f"visitId == {visit_id}"))
```

### Change Data Types in Prepared Data

In [30]:
# | code-fold: false
dtypes_dict = {
    "fullvisitorid": pd.StringDtype(),
    "visitId": pd.StringDtype(),
    "visitNumber": pd.Int8Dtype(),
    "country": pd.StringDtype(),
    "quarter": pd.Int8Dtype(),
    "month": pd.Int8Dtype(),
    "day_of_month": pd.Int8Dtype(),
    "day_of_week": pd.Int8Dtype(),
    "hour": pd.Int8Dtype(),
    "minute": pd.Int8Dtype(),
    "second": pd.Int8Dtype(),
    "source": pd.StringDtype(),
    "medium": pd.StringDtype(),
    "channelGrouping": pd.StringDtype(),
    "hits": pd.Int16Dtype(),
    "bounces": pd.Int16Dtype(),
    "last_action": pd.Int8Dtype(),
    "product_detail_views": pd.Int16Dtype(),
    "promos_displayed": pd.Int16Dtype(),
    "promos_clicked": pd.Int16Dtype(),
    "product_views": pd.Int16Dtype(),
    "product_clicks": pd.Int16Dtype(),
    "pageviews": pd.Int16Dtype(),
    "transact_revenue": pd.Float32Dtype(),
    "time_on_site": pd.Int16Dtype(),
    "browser": pd.StringDtype(),
    "os": pd.StringDtype(),
    "added_to_cart": pd.Int16Dtype(),
    "deviceCategory": pd.StringDtype(),
}

In [31]:
# | code-fold: false
df = df.astype(dtypes_dict)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 92551 entries, 0 to 92858
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   fullvisitorid                  92551 non-null  string        
 1   visitId                        92551 non-null  string        
 2   visitNumber                    92551 non-null  Int8          
 3   visitStartTime                 92551 non-null  datetime64[ns]
 4   country                        92551 non-null  string        
 5   quarter                        92551 non-null  Int8          
 6   month                          92551 non-null  Int8          
 7   day_of_month                   92551 non-null  Int8          
 8   day_of_week                    92551 non-null  Int8          
 9   hour                           92551 non-null  Int8          
 10  minute                         92551 non-null  Int8          
 11  second              

### Separate Columns by Type

The first three rows of the prepared data are shown below

In [32]:
with pd.option_context(
    "display.max_colwidth", None, "display.max_rows", None, "display.max_columns", None
):
    display(df.head(3))

,fullvisitorid,visitId,visitNumber,visitStartTime,country,quarter,month,day_of_month,day_of_week,hour,minute,second,source,medium,channelGrouping,hits,bounces,last_action,product_detail_views,promos_displayed,promos_clicked,product_views,product_clicks,pageviews,transact_revenue,time_on_site,browser,os,deviceCategory,added_to_cart,made_purchase_on_future_visit
0,8163735676529750721,1481176805,1,2016-12-07 22:00:05,United States,4,12,7,4,22,0,5,google,cpc,Paid Search,5,0,0,0,0,0,0,0,5,<NA>,109,Chrome,Windows,desktop,0,False
1,0897634596694862660,1475199828,1,2016-09-29 18:43:48,United States,3,9,29,5,18,43,48,google,organic,Organic Search,21,0,2,2,9,0,90,2,19,<NA>,467,Chrome,Windows,desktop,0,False
2,3891893273235584028,1478844153,1,2016-11-10 22:02:33,United States,4,11,10,5,22,2,33,youtube.com,referral,Social,11,0,2,1,18,0,36,1,10,<NA>,1003,Safari (in-app),iOS,tablet,0,False


Create lists of columns based on their type. Three such lists are shown below

- `datetime`
- categorical
- numerical

In [33]:
# | code-fold: false
datetime_columns = [
    "quarter",
    "month",
    "day_of_month",
    "day_of_week",
    "hour",
]
categorical_columns = [
    "bounces",
    "last_action",
    "source",
    "medium",
    "channelGrouping",
    "browser",
    "os",
    "deviceCategory",
]
numerical_columns = [
    "hits",
    "product_detail_views",
    "promos_displayed",
    "promos_clicked",
    "product_views",
    "product_clicks",
    "pageviews",
    "time_on_site",
    "added_to_cart",
]

### Handling Categorical Columns

High-cardinality categorical features are a problem for machine learning models as they create a large number of dummy variables (after dummy encoding), or a sparse matrix ([1](https://blog.knoldus.com/sparse-matrices-what-makes-them-important-for-machine-learning/), [2](https://www.aiplusinfo.com/blog/what-is-a-sparse-matrix-how-is-it-used-in-machine-learning/)) that slows ML model training. So, it is frequently necessary to reduce this cardinality before training a ML model.

Two of the well-known apprroaches to reduce dimensionality of such features are ([1](https://arxiv.org/abs/2301.12710), [2](https://www.linkedin.com/advice/0/how-do-you-deal-categorical-features-high-cardinality))

1. [frequency endoding](https://towardsdatascience.com/dealing-with-features-that-have-high-cardinality-1c9212d7ff1b)
   - only keep the `N` most common values for each feature and replace all the other (infrequently occurring) values with a placeholder value such as `other`
   - this will be the approach used for the current project
2. class label or target encoding
   - group categorical features by the class labels (the dependent variable, or `y`)

Reducing the cardinality of such features is performed during the data transformation step of a ML workflow. Here, we will demonstrate this before exploratory data analysis (this step) and then apply it during data transformation (next step).

Show the number of unique values in all categorical columns

In [34]:
# | code-fold: false
df_nunique = pd.DataFrame.from_records(
    [{"column": c, "num_unique_values": df[c].nunique()} for c in categorical_columns]
)
df_nunique

,column,num_unique_values
0,bounces,2
1,last_action,7
2,source,116
3,medium,7
4,channelGrouping,8
5,browser,26
6,os,15
7,deviceCategory,3


::: {.callout-tip title="Observations"}

1. High-cardinality categorical columns are present in the training data.
2. The following categorical columns are high-cardinality columns with the largest number of unique values
   - `source` (source of visitor traffic reaching the merchandise store's website)
   - `browser`
   - `os` (visitor's operating system used to access merchandise store's website)

   and will likely need to be binned or grouped
3. Infrequently occurring values in the following medium-cardinality columns related the source of website visitor traffic will also be grouped
   - `channelGrouping`
   - `medium`
4. `last_action` will be left unchanged
   - it is likely that the last action performed by a visitor during their first visit to the merchandise store will have some influence on their probability (propensity) to make a purchase during a future visit
:::

The category distributions (frequencies) after grouping are shown below for all categorical columns (including those that were grouped)

In [35]:
# | code-fold: false
dfs_cats_groups = []
for c in categorical_columns:
    # get fraction of unique values
    df_frequencies = (
        df[c]
        .value_counts()
        .rename("number_of_visitors")
        .to_frame()
        .merge(
            (
                df[c].value_counts(normalize=True).rename("fraction_of_visitors") * 100
            ).to_frame(),
            left_index=True,
            right_index=True,
        )
    )

    # map unique values for last_action and bounces to get meaningful names
    if c == "last_action":
        df_frequencies.index = df_frequencies.index.map(mapper)
    if c == "bounces":
        df_frequencies.index = df_frequencies.index.map({0: False, 1: True})

    # get running total of fraction (cumulative sum)
    df_frequencies = (
        df_frequencies.sort_values(by=["fraction_of_visitors"])
        .assign(
            cumulative_fraction_of_visitors=lambda df: df[
                "fraction_of_visitors"
            ].cumsum(),
            column_name=c,
        )
        .sort_values(by=["fraction_of_visitors"], ascending=False)
    )

    # rename columns
    df_frequencies = df_frequencies.reset_index().rename(columns={c: "column_value"})
    dfs_cats_groups.append(df_frequencies)
df_frequencies_raw = pd.concat(dfs_cats_groups, ignore_index=True)
col = df_frequencies_raw.pop("column_name")
df_frequencies_raw.insert(0, col.name, col)
with pd.option_context("display.max_rows", None):
    display(df_frequencies_raw)

,column_name,column_value,number_of_visitors,fraction_of_visitors,cumulative_fraction_of_visitors
0,bounces,False,65505,70.777193,100.0
1,bounces,True,27046,29.222807,29.222807
2,last_action,Unknown,67511,72.944647,100.0
3,last_action,Product detail views,14731,15.91663,27.055353
4,last_action,Add product(s) to cart,4655,5.029659,11.138724
5,last_action,Completed purchase,3097,3.346263,6.109064
6,last_action,Check out,1635,1.766594,2.762801
7,last_action,Remove product(s) from cart,881,0.951908,0.996207
8,last_action,Click through of product lists,41,0.0443,0.0443
9,source,google,43484,46.983825,100.0


::: {.callout-tip title="Observations"}

1. We'll create frequency groupings as follows
   - `source` and `browser`
     - all categories which occur with a frequency of less than 5% will be grouped into a single value `other`
   - `os`, `channelGrouping` and `medium`
     - all categories which occur with a frequency of less than 10% will be grouped into a single value `other`

   These thresholds were determined by examining the output of `df_frequencies_raw`, which shows the freqencies of all categories for all categorical columns.
:::

Below are lists of categorical columns to be grouped based on this threshold (5% or 10%)

In [36]:
# | code-fold: false
cols_to_group_5_pct = ["source", "browser"]
cols_to_group_10_pct = ["os", "channelGrouping", "medium"]

We'll get names for the columns after grouping, by adding a `_grouped` suffix

In [37]:
# | code-fold: false
grouped_cols_5_pct = [f"{c}_grouped" for c in cols_to_group_5_pct]
grouped_cols_10_pct = [f"{c}_grouped" for c in cols_to_group_10_pct]

Next, create lists of categorical columns that will and will not be grouped and then combine them into a single list

In [38]:
# | code-fold: false
categorical_columns_mapped = (
    # columns that will not be grouped
    list(
        set(categorical_columns) - set(cols_to_group_5_pct) - set(cols_to_group_10_pct)
    )
    # columns that will be grouped
    + grouped_cols_5_pct
    + grouped_cols_10_pct
)

Create a duplicate of the columns that will be grouped and add a `_grouped` suffix to their column name

In [39]:
# | code-fold: false
for c in cols_to_group_5_pct + cols_to_group_10_pct:
    df[f"{c}_grouped"] = df[c]

Finally, perform the grouping using

1. `pandas.value_counts(normalize=True) < 0.05` (5% threshold)
2. `pandas.value_counts(normalize=True) < 0.10` (10% threshold)

where all infrequently occurring values that satisfy these filters will have their values replaced by `other`

In [40]:
# | code-fold: false
df[grouped_cols_5_pct] = df[grouped_cols_5_pct].apply(
    lambda x: x.mask(x.map(x.value_counts(normalize=True)) < 0.05, "other"), axis=0
)
df[grouped_cols_10_pct] = df[grouped_cols_10_pct].apply(
    lambda x: x.mask(x.map(x.value_counts(normalize=True)) < 0.10, "other"), axis=0
)

The cardinality of the columns before and after grouping is shown below

In [41]:
# | code-fold: false
df_nunique.merge(
    pd.DataFrame.from_records(
        [
            {
                "column": c.replace("_grouped", ""),
                "column_grouped": c,
                "num_unique_values_after_grouping": df[c].nunique(),
            }
            for c in categorical_columns_mapped
        ]
    ).assign(column_grouped=lambda df: df["column_grouped"] != df["column"]),
    on=["column"],
    how="left",
)

,column,num_unique_values,column_grouped,num_unique_values_after_grouping
0,bounces,2,False,2
1,last_action,7,False,7
2,source,116,True,5
3,medium,7,True,4
4,channelGrouping,8,True,4
5,browser,26,True,3
6,os,15,True,5
7,deviceCategory,3,False,3


::: {.callout-tip title="Observations"}

1. The cardinality has been significantly reduced for the columns where the infrequently occurring values were grouped (`column_grouped == True`).
2. The cardinality is unchanged for the columns where the infrequently occurring values were not grouped (`column_grouped == False`).
:::

The category distributions (frequencies) after grouping are shown below for all categorical columns (including those that were grouped)

In [42]:
# | code-fold: false
dfs_cats_groups = []
for c in categorical_columns_mapped:
    # get fraction of unique values
    df_frequencies = (
        df[c]
        .value_counts()
        .rename("number_of_visitors")
        .to_frame()
        .merge(
            (
                df[c].value_counts(normalize=True).rename("fraction_of_visitors") * 100
            ).to_frame(),
            left_index=True,
            right_index=True,
        )
    )

    # map unique values for last_action and bounces to get meaningful names
    if c == "last_action":
        df_frequencies.index = df_frequencies.index.map(mapper)
    if c == "bounces":
        df_frequencies.index = df_frequencies.index.map({0: False, 1: True})

    # get running total of fraction (cumulative sum)
    df_frequencies = (
        df_frequencies.sort_values(by=["fraction_of_visitors"])
        .assign(
            cumulative_fraction_of_visitors=lambda df: df[
                "fraction_of_visitors"
            ].cumsum(),
            column_name=c,
        )
        .sort_values(by=["fraction_of_visitors"], ascending=False)
    )

    # rename columns
    df_frequencies = df_frequencies.reset_index().rename(columns={c: "column_value"})
    dfs_cats_groups.append(df_frequencies)
df_frequencies_grouped = pd.concat(dfs_cats_groups, ignore_index=True)
col = df_frequencies_grouped.pop("column_name")
df_frequencies_grouped.insert(0, col.name, col)
with pd.option_context("display.max_rows", None):
    display(df_frequencies_grouped)

,column_name,column_value,number_of_visitors,fraction_of_visitors,cumulative_fraction_of_visitors
0,deviceCategory,desktop,67355,72.776091,100.0
1,deviceCategory,mobile,21790,23.543776,27.223909
2,deviceCategory,tablet,3406,3.680133,3.680133
3,bounces,False,65505,70.777193,100.0
4,bounces,True,27046,29.222807,29.222807
5,last_action,Unknown,67511,72.944647,100.0
6,last_action,Product detail views,14731,15.91663,27.055353
7,last_action,Add product(s) to cart,4655,5.029659,11.138724
8,last_action,Completed purchase,3097,3.346263,6.109064
9,last_action,Check out,1635,1.766594,2.762801


::: {.callout-note title="Notes"}

1. These distributions are shown here after frequency encoding (grouping) the high-cardinality columns in order to determine the thresholds (5% and 10%) for replacing infrequently occurring values in these columns. Earlier, the same was shown in the raw categorical columns. In that `DataFrame`, there were 184 unique categories across all categorical columns (length of `df_frequencies_raw`). After dummy encoding (where we will drop duplicate categories in each raw categorical column - [1](https://towardsdatascience.com/encoding-categorical-variables-one-hot-vs-dummy-encoding-6d5b9c46e2db)), there would be 184 - `<number-of-categorical-columns>` = 184 - 8 = 176 features.
2. After frequency grouping (where we will drop duplicate categories in each grouped categorical column), there are 33 unique categories. After dummy encoding, the number of dummy variables will be 33 - `<number-of-categorical-columns>` = 33 - 8 = 26 features. This frequency encoding approach has reduced the cardinality by (176 - 26) / 176 = 0.86 (or 86%).
:::

The reduction in cardinality of the categorical feaures, after frequency grouping, is calculated below

In [43]:
# | code-fold: false
frac_reduction_in_cats_cardinality = (
    100
    * (
        (len(df_frequencies_raw) - len(categorical_columns))
        - (len(df_frequencies_grouped) - len(categorical_columns))
    )
    / (len(df_frequencies_raw) - len(categorical_columns))
)
print(
    "Frequency encoding (grouping) has reduced cardinality of categorical features by "
    f"{frac_reduction_in_cats_cardinality:,.3f}%"
)

Frequency encoding (grouping) has reduced cardinality of categorical features by 85.795%


The groupings above have been learnt from the training data. We now need to create a lookup table for columns that were grouped so that we can apply the same groupings to unseen data (validation and test data splits). This means when we encounter the same infrequently occurring values in the validation and test data splits, they will be replaced by `other`.

This lookup table is defined below

In [44]:
# | code-fold: false
df_groupings = pd.DataFrame.from_dict(
    {
        c: df[c]
        .value_counts(normalize=True)
        .rename("fraction")
        .to_frame()
        .query(f"fraction < {threshold}")
        .index.tolist()
        for cols, threshold in zip(
            [cols_to_group_5_pct, cols_to_group_10_pct], [0.05, 0.10]
        )
        for c in cols
    },
    orient="index",
).transpose()
display(df_groupings.head())
display(df_groupings.tail())

,source,browser,os,channelGrouping,medium
0,sites.google.com,Firefox,Linux,Social,cpc
1,moma.corp.google.com,Internet Explorer,Chrome OS,Paid Search,affiliate
2,Partners,Opera,(not set),Affiliates,cpm
3,dfa,Edge,Windows Phone,Display,(not set)
4,siliconvalley.about.com,Safari (in-app),Nintendo Wii,(Other),None


,source,browser,os,channelGrouping,medium
107,dailydot.com,None,None,None,None
108,digg.com,None,None,None,None
109,xbidprodmirror.corp.google.com,None,None,None,None
110,seroundtable.com,None,None,None,None
111,spaces.google.com,None,None,None,None


We'll also create a lookup table of unique values in the categorical columns that were not grouped. When we encounter these values in the validation or test data splits, they will remain unchanged.

This lookup table is defined below

In [45]:
# | code-fold: false
df_ungrouped = pd.DataFrame.from_dict(
    {
        c: df[c]
        .value_counts(normalize=True)
        .rename("fraction")
        .to_frame()
        .query(f"fraction >= {threshold}")
        .index.tolist()
        for cols, threshold in zip(
            [cols_to_group_5_pct, cols_to_group_10_pct], [0.05, 0.10]
        )
        for c in cols
    },
    orient="index",
).transpose()
df_ungrouped

,source,browser,os,channelGrouping,medium
0,google,Chrome,Macintosh,Organic Search,organic
1,(direct),Safari,Windows,Direct,referral
2,mall.googleplex.com,None,iOS,Referral,(none)
3,youtube.com,None,Android,None,None


For a quick demonstration of using these two lookup tables, we'll create a dummy validation data `DataFrame` below with two categorical features

In [46]:
df_val = pd.DataFrame.from_records(
    [
        {"source": "Partners", "browser": "Internet Explorer"},
        {"source": "dfa", "browser": "new-browser"},
        {"source": "new-source-value", "browser": "Chrome"},
    ]
)
df_val

,source,browser
0,Partners,Internet Explorer
1,dfa,new-browser
2,new-source-value,Chrome


We'll now apply both the lookup tables defined above using the following approach

1. for all columns that were grouped, create columns with a suffix `_grouped` which contains the value `other` for infrequently occurring values
2. for all columns that were not grouped, create columns with a suffix `_ungrouped` which contains the same values with no changes
3. combine columns with the `_ungrouped` and `_grouped` suffixes into a single column column
   - to do this, fill missing values in the `_grouped` column with those in the `_ungrouped` column
4. drop original columns and rename the combined columns appropriately

In [47]:
# | code-fold: false
categorical_columns_validation_data = ["source", "browser"]
for c in categorical_columns_validation_data:
    # 1. replace infrequent values in columns that were grouped (add suffix _grouped)
    df_val[f"{c}_grouped"] = df_val[c].map(
        {c_grouped: "other" for c_grouped in df_groupings[c].tolist()}
    )
    # 2. keep all values in columns that were not grouped (add suffix _ungrouped)
    df_val[f"{c}_ungrouped"] = df_val[c].map(
        {c_ungrouped: c_ungrouped for c_ungrouped in df_ungrouped[c].tolist()}
    )
    # 3. combine columns that were replaced (_grouped) and those that were not replaced (_ungrouped)
    df_val[f"{c}_grouped"] = df_val[f"{c}_grouped"].fillna(df_val[f"{c}_ungrouped"])
# 4. drop unwanted columns and rename
df_val = df_val.drop(
    columns=["browser_ungrouped", "source_ungrouped"]
    + categorical_columns_validation_data
).rename(columns={f"{c}_grouped": c for c in categorical_columns_validation_data})
df_val

,source,browser
0,other,other
1,other,NaN
2,NaN,Chrome


::: {.callout-tip title="Observations"}

1. In both features of the validation data, there are new categories that were not seen in the training data. After applying the two lookup tables above, these values are replaced by `None`s. We can fill these missing values using
   - `new` (or keep it as `None`) to indicate this is a new category
     - the ML model has not seen this value in the appropriate feature during training, so the predictive power of such a feature in the unseen (validation) data will likely be reduced or minimal (the model won't know its relationship to the label `y`)
   - `other` to group this into the infrequently occurring categories that were identified from the training data
     - the disadvantage is that these new categories might have a different relationship to the label label (`y`) than the grouped (`other`) category
     - in such a scenario
       - the ML model might not able to leverage the full predictive power of such new categories in the validation (unseen) data when it makes predictions since it was not trained to learn this relationship in the training data
       - the model will make predictions based on the relationship learnt between the grouped (`other`) category and the label (`y`)
:::

During data processing (after this EDA step), we will create the training, validation and test data splits for ML development and the same workflow will be used to handle categorical features during data processing.

## Key Findings

### Nested Attributes of First-Time Visits

1. Promotion nested column
   - When `view_promo = True`, a visitor has viewed a promotion. If it is not viewed, then it is `None`
   - When a visitor clicks a promotion after viewing it
     - `click_promo = True`
     - `view_promo is NULL`
2. Product nested column
   - if a product is viewed in a listing (`product_count > 0`) during a visit, then there are only two possible values for `clicked_product` and `viewed_product`, namely `True` and `None`
   - a product is not viewed in a listing, then the only value in these same two columns is `None`

### Recommendations for Data Processing

1. Negligible duplicates exist for
   - `visitId`
     - the reason for this duplication is known
     - such duplicates are kept in the data
   - `fullvisitorid`
     - the reason for this duplication is not known
     - all but the first of such duplicates should be dropped
2. The cardinality across the combination of all categorical columns extracted from visitors' first visit to the store can be significantly reduced by replacing infrequently occuring values by the category `other`
   - `source` and `browser` columns
     - all categories which occur with a frequency of less than 5% should be grouped into a single value `other`
   - `os`, `channelGrouping` and `medium` columns
     - all categories which occur with a frequency of less than 10% should be grouped into a single value `other`

## Summary of Assumptions

1. Negligible duplicates in `fullvisitorid` are found and are not well understood and so they should be dropped in the training, validation and test data splits during data transformation.

## Summary of Tasks Performed

This step has performed the following

1. extracted attributes from dataset to create a *prepared dataset* for use in EDA
   - flattened nested columns for products and promotions
   - extracted columns that should intuitively help predict probability of making a purchase on a return (future) visit
2. addressed duplicated visits
3. handled high-cardinality categorical columns

## Limitations

None.

## Next Step

The next step will be to perform exploratory data analysis using data prepared using the findings from this data preparation step.